#  **BUILD 2025 LONDON - SYNTHETIC DATA GENERATION FOR DEVELOPMENT AND TESTING OF ML MODELS**
### Notebook - Main
---
### What We'll Do:
1. **Data Ingestion**: Fetch customer and claims data from our database
2. **Synthetic Data Generation**: Utilize both `cortex.complete` and `GENERATE_SYNTHETIC_DATA` procedure to create synthetic data
2. **Data Transformation**: Utilize Snowpark DataFrames for data preparation and analysis
3. **Model Training**: Train a XGB Classifier model
4. **Model Registry**: Saving the model to Snowflake Model Registry

Remember to add the necessary packages in the 'Packages' drop down at the top. For example,
- `snowflake-snowpark-python`
- `snowflake-ml-python`
- `tabulate`
- `seaborn`

In [ ]:
from snowflake.snowpark import Window
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import LongType
from snowflake.ml.modeling.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.modeling.metrics import *
# from snowflake.ml.registry import Registry

# Import python packages
import streamlit as st
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import matplotlib.pyplot as plt
import seaborn as sns
import json
# import tabulate
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
snowflake_environment = session.sql('select current_user(), current_version()').collect()
from snowflake.snowpark.version import VERSION
from snowflake.ml import version

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(VERSION[0],VERSION[1],VERSION[2]))
print('Snowflake ML version        : {}.{}.{}'.format(version.VERSION[0],version.VERSION[2],version.VERSION[4]))

In [ ]:
claim_data = session.read.table("BUILD_LONDON_25.DATA.CLAIM_DATA")
customer_data = session.read.table("BUILD_LONDON_25.DATA.CUSTOMER_DATA")

st.dataframe(claim_data.limit(50))
st.dataframe(customer_data.limit(50))

In [ ]:
-- SELECT SNOWFLAKE.CORTEX.COMPLETE('mixtral-8x7b',
-- ' Generate a structured dataset of fake but realistic insurance claims. The dataset should include diverse claim types, varying amounts, and different levels of complexity. The output should be in a tabular format with the following columns:

-- Claim ID (Unique alphanumeric identifier)
-- Policyholder Name (Realistic but randomly generated names)
-- Policy Number (Unique identifier)
-- Claim Type (Auto, Home, Health, Life, Disability, etc.)
-- Claim Date (Random date within the last five years)
-- Incident Description (Concise but detailed description of the claim)
-- Claim Amount (Varied realistic claim amounts based on claim type)
-- Status (Open, Closed, Under Investigation, Denied)
-- Adjuster Name (Randomly generated name)
-- Payout Amount (0 if denied, realistic value if approved)
-- Fraud Flag (Yes/No, randomly assigned with logical probability)
-- Ensure the dataset contains a variety of claim types, realistic descriptions, and varying monetary values. The data should mimic real-world patterns, including fraudulent claims, high-value claims, and small routine claims. Provide at least 100 entries.')

In [ ]:
# from snowflake.cortex import Complete

# #llm = 'llama3.2-3b'
# llm = 'claude-3-5-sonnet'
# prompt = f"""
# Given the schema of the dataframe: {str(customer_data.schema)}
# A sample of the dataframe: {customer_data.sample(n=10).to_pandas().to_markdown()}. 
# Produce a synthetic dataset that follows the same structure as the schema and the provided sample data.
# Provide 10 rows of these data.
# In the output, do not include any descriptions. Only output the list of dictionary with column name as the key and synthetic value generated value as value."""
# llm_response = Complete(llm, prompt)
# print(llm_response)

In [ ]:
CALL SNOWFLAKE.DATA_PRIVACY.GENERATE_SYNTHETIC_DATA({
    'datasets':[
        {
          'input_table': 'BUILD_LONDON_25.DATA.claim_data',
          'output_table': 'BUILD_LONDON_25.DATA.claim_data_synthetic',
          'columns': {'policy_number': {'join_key': True}}
        },
        {
          'input_table': 'BUILD_LONDON_25.DATA.customer_data',
          'output_table': 'BUILD_LONDON_25.DATA.customer_data_synthetic',
          'columns' : {'policy_number': {'join_key': True}}

        }
      ],
      'replace_output_tables':True
  });

In [ ]:
select * from BUILD_LONDON_25.DATA.claim_data_synthetic limit 20;

In [ ]:
claim_data_synthetic = session.read.table("BUILD_LONDON_25.DATA.CLAIM_DATA_SYNTHETIC").to_pandas()
customer_data_synthetic = session.read.table("BUILD_LONDON_25.DATA.CUSTOMER_DATA_SYNTHETIC").to_pandas()

synthetic_data = pd.merge(how='inner', left=claim_data_synthetic, right=customer_data_synthetic, on='POLICY_NUMBER')
synthetic_data['IS_SYNTHETIC'] = 1


data = pd.merge(how='inner', left=claim_data.to_pandas(), right=customer_data.to_pandas(), on='POLICY_NUMBER')
data['IS_SYNTHETIC'] = 0

training_data_df_pd = pd.concat([synthetic_data,data])


In [ ]:
training_data_df_pd

In [ ]:
training_data_df = session.create_dataframe(training_data_df_pd)

training_data_df = training_data_df.replace('?', None)

mode_value_report = training_data_df.select(mode(col("POLICE_REPORT_AVAILABLE"))).collect()[0][0]
mode_value_authority = training_data_df.select(mode(col("AUTHORITIES_CONTACTED"))).collect()[0][0]
training_data_df = training_data_df.with_column("POLICE_REPORT_AVAILABLE", 
    when(col("POLICE_REPORT_AVAILABLE").is_null(), mode_value_report)
    .otherwise(col("POLICE_REPORT_AVAILABLE")))

training_data_df = training_data_df.with_column("AUTHORITIES_CONTACTED", 
    when(col("AUTHORITIES_CONTACTED")=='0', mode_value_authority)
    .otherwise(col("AUTHORITIES_CONTACTED")))

In [ ]:
import seaborn as sns
pandas_df = training_data_df.to_pandas()
sns.displot(pandas_df, x="CLAIM_AMOUNT", hue = "IS_SYNTHETIC", element="step")

In [ ]:
# Due to the high correlation between Age and policy_length_month, let's drop age.
# Let's all drop the date fields 
training_data_df = training_data_df.drop(["AGE", "INCIDENT_DATE", "POLICY_START_DATE"])

In [ ]:
from snowflake.snowpark.functions import col

training_data_df = training_data_df.with_column("FRAUD_REPORTED", col("FRAUD_REPORTED").astype(LongType()))
train_data, test_data = training_data_df.random_split(weights = [0.8, 0.2], seed = 43)

In [ ]:
# Define the categories with their specific order
categories = {
    "INSURED_EDUCATION_LEVEL": np.array(["High School", "Associate", "College", "Masters", "JD", "MD", "PhD"]),
    "INCIDENT_SEVERITY": np.array(["Trivial Damage", "Minor Damage", "Major Damage", "Total Loss"])
}
# Create the OrdinalEncoder with specified categories
OrdinalEncoding = OrdinalEncoder(
    input_cols=["INSURED_EDUCATION_LEVEL", "INCIDENT_SEVERITY"],
    output_cols=["INSURED_EDUCATION_LEVEL_OE", "INCIDENT_SEVERITY_OE"],
    categories=categories,
    handle_unknown="use_encoded_value",
    unknown_value=-1,
    drop_input_cols=True
)

# Define the columns to encode
columns_to_encode = [
    "INSURED_SEX",
    "INSURED_OCCUPATION",
    "INCIDENT_TYPE",
    "AUTHORITIES_CONTACTED",
    "POLICE_REPORT_AVAILABLE"
]
# Create a OneHotEncoder instance
OneHotEncoding = OneHotEncoder(
    input_cols=columns_to_encode,
    output_cols=[f"{col}_encoded" for col in columns_to_encode],
    drop_input_cols=True,  # Keep original columns
    handle_unknown='ignore'  # Ignore any unknown categories during transform
)

# Define the columns to scale
columns_to_scale = [
    'POLICY_LENGTH_MONTH',
    'POLICY_DEDUCTABLE',
    'POLICY_ANNUAL_PREMIUM',
    'CLAIM_AMOUNT'
]
# Create the StandardScaler
StandardScaling = StandardScaler(
    input_cols=columns_to_scale,
    output_cols=[f"{col}_SCALED" for col in columns_to_scale],
    with_mean=True,
    with_std=True,
    drop_input_cols=True  # Keep original columns
)

# Determine the label column name
# feature_columns = train_data.columns.remove('FRAUD_REPORTED_LONG')
label_column = ['FRAUD_REPORTED']
output_column = ['PREDICTED_FRAUD']


# # Initially, we can run this under the XGB Classifier model. However, you will notice that
# # the model overfits on the training data and performs poorly on the test dataset
# xgbmodel = XGBClassifier(
#     random_state=1, 
#     #input_cols=feature_columns,    #here we are passing all columns so we have commented out. If you have specific columns set as features, you should specify them here
#     label_cols=label_column,
#     output_cols=output_column
#     )


xgb_grid_search = GridSearchCV(
    estimator=XGBClassifier(),
    param_grid={
        "n_estimators":[10, 20],
        "max_depth": [4],
        "learning_rate":[0.1],
    },
    n_jobs = -1,
    #input_cols=feature_columns,    #here we are passing all columns so we have commented out. 
                                    #If you have specific columns set as features, you should specify them here
    label_cols=label_column,
    output_cols=output_column,
)


model_pipeline = Pipeline(
    steps=[
        ("Ordinal_encoding",OrdinalEncoding),
        ("OneHotEncoding",OneHotEncoding),
        ("standardscaler",StandardScaling),
        ("CV_XGBClassifier", xgb_grid_search)
    ]
)

In [ ]:
def train_model(train_data,test_data):
    
    xgb_gs_fitted_training = model_pipeline.fit(train_data)
    xgb_gs_train = xgb_gs_fitted_training.predict(train_data)
    xgb_gs_predictions = xgb_gs_fitted_training.predict(test_data)

    # Let's start with the basic metric, Accuracy, which the number of correct predictions made divided by the total number of predictions made,
    ACCURACY = accuracy_score(df=xgb_gs_predictions, y_true_col_names=label_column, y_pred_col_names=output_column)
    
    # RPC AUC is slightly perferred IMO. Anything above 50% or .5 is better than random guessing
    AUC = roc_auc_score(df=xgb_gs_predictions, y_true_col_names=label_column, y_score_col_names=output_column)

    return ACCURACY, AUC

In [ ]:
wh = str(session.get_current_warehouse()).strip('"')
print(f"Current warehouse: {wh}")
print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

session.sql(f"alter warehouse {session.get_current_warehouse()} set WAREHOUSE_SIZE = LARGE WAIT_FOR_COMPLETION = TRUE").collect()

print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

In [ ]:
model_all_data = train_model(train_data,test_data)

In [ ]:
train_data_real = train_data.filter(col('IS_SYNTHETIC')==0)
model_real_data = train_model(train_data_real,test_data)

In [ ]:
print('Performance with real data only')
print(f"AUC: {model_real_data[1]:.4f}")

print('-'*50)

print('Performance with real data AND synethtic data')
print(f"AUC: {model_all_data[1]:.4f}")

In [ ]:
wh = str(session.get_current_warehouse()).strip('"')
print(f"Current warehouse: {wh}")
print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

session.sql(f"alter warehouse {session.get_current_warehouse()} set WAREHOUSE_SIZE = XSMALL WAIT_FOR_COMPLETION = TRUE").collect()

print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",model_pipeline.to_sklearn().named_steps['CV_XGBClassifier'].best_estimator_)
print("\n The best score across ALL searched params:\n",model_pipeline.to_sklearn().named_steps['CV_XGBClassifier'].best_score_)
print("\n The best parameters across ALL searched params:\n",model_pipeline.to_sklearn().named_steps['CV_XGBClassifier'].best_params_)

In [ ]:
# Let's now register the CV Classfier model into the model_registry
Reg = Registry(
    session=session,
    database_name=session.get_current_database(),
    schema_name='data',
)

In [ ]:
model_name = 'XGB_GS_FRAUD_MODEL'
model_version = get_next_version(Reg, model_name)

mv = Reg.log_model(xgb_gs_fitted_training,
    model_name=model_name,
    version_name=model_version,
    conda_dependencies=["snowflake-ml-python"],
    comment="Model trained using GridsearchCV in Snowpark to predict fraud claims",
    #metrics={"Acc": ACCURACY, "AUC": AUC}, # We can save our model metrics here
    options= {"relax_version": False}
)

m = Reg.get_model(model_name)
m.default = model_version